In [1]:
!pip install dash

In [10]:
#masukkan library yang dibutuhkan
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from sklearn.datasets import load_iris

#memuat dataset iris dari scikit-learn kemudian diubah ke dataframe dan mengkasih nama spesies warna
data_iris = load_iris()
dataframe_iris = pd.DataFrame(data = data_iris.data, columns = data_iris.feature_names)
dataframe_iris["Species"] = data_iris.target_names[data_iris.target]
warna_spesies = {"setosa" : "purple", "versicolor": "yellow", "virginica": "cyan"}

#inisialisasi apk dash
aplikasi = dash.Dash(__name__)

#membuat layout untuk ditampilan dashboardnya dan memilih filter x dan y kemudian jumlah kelas spesies
aplikasi.layout = html.Div([
    html.H1("Dashboard Dari Iris Datasetnya"),
    
    html.Div([
        html.Label("Variabel X"),
        dcc.Dropdown(
            id = "x feature dropdown",
            options = [{"label": feature, "value": feature} for feature in dataframe_iris.columns[:-1]],
            value = dataframe_iris.columns[0], 
            multi = False,
            style = {"width": "50%"}  
        ),
    ]),
    
    html.Div([
        html.Label("Variabel Y"),  
        dcc.Dropdown(
            id = "y feature dropdown",
            options = [{"label": feature, "value": feature} for feature in dataframe_iris.columns[:-1]],
            value = dataframe_iris.columns[1], 
            multi = False,
            style = {"width": "50%"}  
        ),
    ]),
    
    dcc.Dropdown(
        id = "species dropdown",
        options = [{"label": feature, "value": feature} for feature in dataframe_iris["Species"].unique()],
        value = [dataframe_iris["Species"].unique()[0]], 
        multi = True,
        style = {"width": "50%"}  
    ),
   
#plot dari dropdown
    dcc.Graph(id = "feature plot")
])
  
#manggil kembali untuk memperbarui label variabelnya
@aplikasi.callback(
    Output("feature plot", "figure"),
    [Input("x feature dropdown", "value"),
     Input("y feature dropdown", "value"),
     Input("species dropdown", "value")]
)

def update_plot(fitur_x, fitur_y, pilih_spesies):
    # Membuat plot berdasarkan fitur X, Y, dan spesies yang dipilih
    plot_data = [
        {
            "x": dataframe_iris[(dataframe_iris["Species"] == species) & (dataframe_iris["Species"].isin(pilih_spesies))][fitur_x],
            'y': dataframe_iris[(dataframe_iris["Species"] == species) & (dataframe_iris["Species"].isin(pilih_spesies))][fitur_y],
            "type": "scatter",
            "mode": "markers",
            "marker": {"color": warna_spesies[species]},
            "name": species
            } for species in pilih_spesies
        ] 
    fig = {
        "data" : plot_data,
        "layout": {
            "title": f"Perbandingan dari {fitur_x} dan {fitur_y} untuk spesies dari dataset iris {','.join(pilih_spesies)}",
            "x axis": {'title': fitur_x},
            "y axis": {'title': fitur_y}
        }
    }
    return fig

# Menjalankan aplikasi Dashboardnya
if __name__ == "__main__":
    aplikasi.run_server(debug=True)